<a href="https://colab.research.google.com/github/GasanaElysee12/Lab3_n_gram_NLP/blob/main/3rd_lab_for_n_gram_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">AMMI NLP - Review sessions</h1>

<h1 style="font-family:verdana;font-size:180%;text-align:Center;color:#993333"> Lab 3: n-gram models </h1>

**Big thanks to Amr Khalifa who improved this lab and made it to a Jupyter Notebook!**

In [ ]:
import io, sys, math, re
from collections import defaultdict
import numpy as np

In [ ]:
# data_loader

def load_data(filename):
    '''
    parameters:
    filename (string): datafile
    
    Returns:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    '''
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    vocab = defaultdict(lambda:0)
    for line in fin:
        sentence = line.split()
        data.append(sentence)
        for word in sentence:
            vocab[word] += 1
    return data, vocab

In [ ]:

print("load training set..")
print("\n")
train_data, vocab = load_data("train1.txt")
print(train_data[0])
print("\n")
print("how :",vocab['how'])
print("load validation set")
valid_data, _ = load_data("valid1.txt")


load training set..


['<s>', 'my', 'fathers', "don't", 'speak', 'dutch.', '</s>']


how : 107
load validation set


In [ ]:
def remove_rare_words(data, vocab, mincount = 1):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    mincount(int): the minimum count 
    
    Returns: 
    data_with_unk(list of lists): data after replacing rare words with <unk> token
    '''
    # replace words in data that are not in the vocab 
    # or have a count that is below mincount
    data_with_unk = []

    ## FILL CODE
    #Here we need two for loops, one for accessing sentence in list of list and other one for word in sentence
    for sentence in data:
        # new sentence to store also replaced word by <unk>
        update_sentence = []
        for word in sentence:
            #check whether the word is in the vocad keys and also the value count is high than mincount 
            if word in vocab and vocab[word]>mincount:
                #once condition is true, store those words.
                update_sentence.append(word)
                # if word is there but less or equal min count replace by unk and store it also.
            else:
                update_sentence.append('<unk>')
            #After that keep the sendtence in the collection of all sentences which is data_with_unk.
        data_with_unk.append(update_sentence)
        #output the collection of all sentences that have been updated.
    return data_with_unk

In [ ]:
print("remove rare words")
#Remove the word with number of count which is less than or equal to minicount which is one here in training
#validation data.
train_data = remove_rare_words(train_data, vocab, mincount = 1)
valid_data = remove_rare_words(valid_data, vocab, mincount = 1)
#  Check the first sentence in the train_data, here you might check anyone you want to make 
# sure that method is working
print(train_data[0])

remove rare words
['<s>', 'my', '<unk>', "don't", 'speak', '<unk>', '</s>']


In [ ]:
# Build the model to produce the n_gram and probability
def build_ngram(data, n):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    n (int): size of the n-gram
    
    Returns:
    prob (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    '''
    #Set the number of word as zero.
    words_amounts = 0
    # Dictionary of dictionary to keep context which is n_gram and as key. Then inside next words to predict
    #will be the also key of internal dictionary with values as their probabilities.
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))
    #Access the sentence in the list of sentences.
    for sentence in data:
        #Make sentence as tuple since we need to keep ngram as key, in python, tuple can be a key. that is why.
        sentence = tuple(sentence)

        for i in range(len(sentence)):
            #We need to calculate total number of words in a sentence.
                words_amounts +=1
                # we need to populate our words in dictionary as ngram
                for length in range(n):
                    #To know difference between a given word in sentence and taken on in ngram if we didn't take many
                    #if is true process must stop, we to go in history based on number of grams (n)
                    if i-length < 0:
                        break
                    #If condition above happen it will be ignored and go to the next word.
                    #It will happen for N=0,N=1, and N=3 
                    counts[sentence[i-length:i]][sentence[i]] +=1 
                            
        #Building of probability dictionary.
    probability_dict  = defaultdict(lambda: defaultdict(lambda: 0.0))
    # Build the probabilities from the counts
    # Be careful with how you normalize!
    
    # context as n_gram, now we are going to build based on ngram as keyword in count dictionary
    for context in counts.keys():
    ## FILL CODE
    
    #Initialize the denominator of probability
        probability_denominator =0
        
        #Now for the last word  to be predicted which is in ngram (context), here it is a value for of context.
        for word in counts[context].keys():
            # if it is available the number of count of this word will be increased to denominator
            probability_denominator += counts[context][word]
            # then for that word we are going to compute its probability based on its count together with context.
        for word in counts[context].keys():
            # counts[context][word]:is the total number of time word appeared together with ngram(context)
            #divide it with probability denominator to get probability of being the next word given context.
            #Then store that word as value of context(ngram) and its value be corresponding probability.
            probability_dict[context][word] = counts[context][word]/probability_denominator 
      #return the dictionary which contain ngram, next words to be selected and correponding probabilities.
    return probability_dict

In [ ]:
# RUN TO BUILD NGRAM MODEL

#If the number of gram is 2, we can see the next word predicted and corresponding probability in 
n = 2
print("build ngram model with n = ", n)
model = build_ngram(train_data, n)

In [ ]:
#Down you  can see
list_=[]
for key, value in model.items():
    list_.append(list(value))
    print(list_)

Here, implement a recursive function over shorter and shorter context to compute a "stupid backoff model". An interpolation model can also be implemented this way.

In [ ]:
def get_prob(model, context, w):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    context (list of strings): a sentence
    w(string): the word we need to find it's probability given the context
    
    Retunrs:
    prob(float): probability of this word given the context 
    '''

    # code a recursive function over 
    # smaller and smaller context
    # to compute the backoff model
    
    ## FILL CODE

    #check the whether the ngram(context) is in probability dictionary and the word to be predicted
    
    if context in model and word in model[context]:
      #if the word is in the value of context return its probabilty
        return model[context][word]
        #once not available, create the new small ngram and check probability for that word
    else:
        return 0.4*get_prob(model, context[1:], word)

    return 0.0


In [ ]:
def perplexity(model, data, n):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    data (list of lists): each list is a sentence of the text
    n(int): size of the n-gram
    
    Retunrs:
    perp(float): the perplexity of the model 
    '''
    ## FILL CODE
    #initialize perplexity as 0.0 ans total length of sequence,T as 0.
    perp, sequence_length = 0.0, 0
    for sentence in data:
      #make a sentence as tuple in order to create ngram as tuple
        sentence = tuple(sentence)
        for i in range(1, len(sentence)):
          #number of context in past and current we take the one with minimum number
            number_context = min(n-1, i)
            #then formulate context
            context = sentence[i-number_context:i]
            #update the perplexity by computing log to minimize space and also we are multiplying small value
            perp += -math.log(get_prob(model, context, sentence[i]))
            # After we update the length of sequence
            sequence_length += 1
            #  Find the perplexity
    perp = math.exp(perp/sequence_length)
    return perp

In [ ]:
# COMPUTE PERPLEXITY ON VALIDATION SET
#Here we might set any value of n grams to see perplexity.

print("The perplexity is", perplexity(model, valid_data, n=n))

The perplexity is 1.0


In [ ]:
#Here we might set any value of n grams to see perplexity.
# if n=3
n=3
print("The perplexity is", perplexity(model, valid_data, n=n))

In [ ]:
def get_proba_distrib(model, context):
    ## need to get the the words after the context and their probability of appearance
    ## after this context 
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    context (list of strings): the sentence we need to find the words after it and 
    thier probabilites
    
    Retunrs:
    words_and_probs(dic): {word: probability of word given context}
    
    '''
    # code a recursive function over context
    # to find the longest available ngram
    
    ## FILL CODE
    # Here if n gram is in the model, we ajre going to get n_gram(context)
    if context in model:
        return model[context]
        #If is not there we create the new small n_gram context[1:] in order to generate its probability.
    else:
        return get_proba_distrib(model, context[1:])

In [ ]:
def generate(model):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    
    Retunrs:
    sentence (list of strings): a sentence sampled according to the language model. 
    

    '''
    # generate a sentence. A sentence starts with a <s> and ends with a </s>
    # Possiblly a use function is:
    # np.random.choice(x, 1, p = y)

    # where x is a list of things to sample from
    # and y is a list of probability (of the same length as x)



    #If sentence is ["<s>"]
    sentence = ["<s>"]
    #As we know that the end of the sentence is </s>, once the end of sentence is not </s> and we have small sentence of length<100.
    while sentence[-1] != "</s>" and len(sentence)<100:
        ## FILL CODE
        # this tell us that we can create a ngram in that sentence and check the probability.
        probability = get_proba_distrib(model, tuple(sentence))
        #By using random.choice, it will give us key which is word and (1) means that we need one and get the corresponding probability.
        word_and_probability= np.random.choice((list(probability.keys())), 1, p = list(probability.values()))
        # keep word in sentence as it is at index 0.
        sentence.append(word_and_probability[0])
        # Then we check those predicted words
    return sentence

In [ ]:
# GENERATE A SENTENCE FROM THE MODEL

# Now we can print the word generated to see what are them
print("Generated sentence: ",generate(model))

Generated sentence:  ['<s>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']


Once you are done implementing the model, evaluation and generation code, you can try changing the value of `n`, and play with a larger training set (`train2.txt` and `valid2.txt`). You can also try to implement an interpolation model.

## Let work on the train2.txt and valid2.txt

In [ ]:
print("load training set..")
print("\n")
train_data, vocab = load_data("train2.txt")
print(train_data[0])
print("\n")
print("how :",vocab['how'])
print("load validation set")
valid_data, _ = load_data("valid2.txt")

#Remove rare word by replacing with unknown.

In [ ]:


train_data = remove_rare_words(train_data, vocab, mincount = 1)
valid_data = remove_rare_words(valid_data, vocab, mincount = 1)


# Let generate the probability for the words in train2.txt and valid2.txt, for n=3

In [ ]:

n = 3
print("build ngram model with n = ", n)
model = build_ngram(train_data, n)


# Let generate the probability for the words in train2.txt and valid2.txt, for n=4

In [ ]:

n = 4
print("build ngram model with n = ", n)
model = build_ngram(train_data, n)

#Here we might set any value of n grams to see perplexity. Let's set n=2,

In [ ]:


print("The perplexity is", perplexity(model, valid_data, n=n))

#Here we might set any value of n grams to see perplexity. Let's set n=3.

In [ ]:

n=3
print("The perplexity is", perplexity(model, valid_data, n=n))

# Now we can print the word generated to see what are them

In [ ]:



print("Generated sentence: ",generate(model))

In [ ]:
def interpolation(data,n):

  prob=0.0
  for sentence in data:
   dic=build_ngram(sentence, n)
   lambda1=0.3,lambda2=0.5,lambda3=0.2
   for context in dic.keys():
    if len(context)==1 or len(context)==2:
      for word in dic[context].values():
        prob +=dic[context][word]+